In [5]:
import pandas as pd
import numpy as np

In [12]:

#read from flat excel waste per category
df1_waste_cate = pd.read_excel('../source/waste_cate.xlsx', 'Sheet 1') 
df2_waste_cate = pd.read_excel('../source/waste_cate.xlsx', 'Sheet 2')
#read from flat excel food waste 
df1_food_waste = pd.read_excel('../source/food_waste.xlsx','Sheet 1')
df2_food_waste  = pd.read_excel('../source/food_waste.xlsx','Sheet 2')
df3_food_waste  = pd.read_excel('../source/food_waste.xlsx','Sheet 3')
df4_food_waste  = pd.read_excel('../source/food_waste.xlsx','Sheet 4')

#read from text file 
df_composition = pd.read_csv('../source/food_composition.csv')

#read from bigquery csv
df_bq_keywords = pd.read_csv('../source/bq_keywords_food_waste.csv')


#read from API csv
df_api_12 = pd.read_csv('../source/api_data12.csv')
df_api_12_3 = pd.read_csv('../source/api_data12_3.csv')

df_top_fruit_veg_wasted = pd.read_csv('../source/top_fruit_veg_wasted.csv')




c:\Users\krant\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\krant\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\krant\AppData\Local\Temp\ipykernel_17236\4032773392.py:14: DtypeWarning: Columns (9,11,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_bq_keywords = pd.read_csv('../source/bq_keywords_food_waste.csv')


In [27]:

eu_country = ['Albania',
'Austria',
'Belgium',
'Bosnia and Herzegovina',
'Bulgaria',
'Croatia',
'Cyprus',
'Czechia',
'Denmark',
'Estonia',
'Finland',
'France',
'Germany',
'Greece',
'Hungary',
'Iceland',
'Ireland',
'Italy',
'Latvia',
'Lithuania',
'Luxembourg',
'Malta',
'Montenegro',
'Netherlands (Kingdom of the)',
'North Macedonia',
'Norway',
'Poland',
'Portugal',
'Romania',
'Serbia',
'Slovakia',
'Slovenia',
'Spain',
'Sweden',
'Switzerland',
'TÃ¼rkiye',
'United Kingdom of Great Britain and Northern Ireland','Netherlands Antilles [former]','Kosovo']


In [73]:
#read from flat excel waste per category
 

# Slice rows from index 8 to 48 of df1_waste_cate and reset the index, dropping the old index
df_waste = df1_waste_cate[11:49].reset_index(drop=True)

# Rename the columns of df_waste using the values from the 9th row of df1_waste_cate
df_waste = df_waste.rename(columns=df1_waste_cate.iloc[7])

# Add a new column named 'year' to df_waste and set all values in that column to 2020
df_waste['year'] = 2020

# Slice rows from index 8 to 48 of df2_waste_cate and reset the index, dropping the old index
df_waste2 = df2_waste_cate[11:49].reset_index(drop=True)

# Rename the columns of df_waste2 using the values from the 9th row of df2_waste_cate
df_waste2 = df_waste2.rename(columns=df2_waste_cate.iloc[7])

# Add a new column named 'year' to df_waste2 and set all values in that column to 2021
df_waste2['year'] = 2021

# Concatenate df_waste and df_waste2 vertically, stack them on top of each other, and reset the index, dropping the old index
df_waste = pd.concat([df_waste, df_waste2]).reset_index(drop=True)

df_waste

# First, find the column names that are NaN
nan_columns = df_waste.columns[df_waste.columns.isna()]

# Then drop these columns
df_waste_c = df_waste.drop(columns=nan_columns)

df_waste_c = df_waste_c.rename(columns={'COICOP (Labels)':'country'})
df_waste_c.columns = [col.lower().replace(' ', '_') for col in df_waste_c.columns]


df_waste_c['id'] = df_waste_c.apply(lambda row: f'waste_cat_{row["country"].lower()}{row["year"]}', axis=1)
# Assuming 'df_waste_c' is your DataFrame and 'column_to_move' is the name of the column you want to move to the first position
column_to_move = 'id'  # Adjust this to the actual name of the column you want to move

# Extract the column you want to move
moved_column = df_waste_c[column_to_move]

# Drop the column from its original position
df_waste_c = df_waste_c.drop(columns=[column_to_move])

# Insert the column at the first position
df_waste_c.insert(0, column_to_move, moved_column)
# Insert the column at the first position

# df_waste_c.to_csv('../source/check.csv',index=False)
# df_waste_c


In [70]:
# Slice rows from index 12 to 38 of df1_food_waste and reset the index, dropping the old index
df_kilo_capita = df1_food_waste[12:39].reset_index(drop=True)

# Rename the columns of df_kilo_capita using the values from the 9th row of df1_food_waste
df_kilo_capita = df_kilo_capita.rename(columns=df1_food_waste.iloc[9])

# Add a new column named 'year' to df_kilo_capita and set all values in that column to 2020
df_kilo_capita['year'] = 2020

# Slice rows from index 12 to 38 of df2_food_waste and reset the index, dropping the old index
df_kilo_capita2 = df2_food_waste[12:39].reset_index(drop=True)

# Rename the columns of df_kilo_capita2 using the values from the 9th row of df2_food_waste
df_kilo_capita2 = df_kilo_capita2.rename(columns=df2_food_waste.iloc[9])

# Add a new column named 'year' to df_kilo_capita2 and set all values in that column to 2021
df_kilo_capita2['year'] = 2021

# Concatenate df_kilo_capita and df_kilo_capita2 vertically, stack them on top of each other, and reset the index, dropping the old index
df_kilo_capita = pd.concat([df_kilo_capita, df_kilo_capita2]).reset_index(drop=True)

# First, find the column names that are NaN
nan_columns = df_kilo_capita.columns[df_kilo_capita.columns.isna()]

# Then drop these columns
df_kilo_capita_c = df_kilo_capita.drop(columns=nan_columns)
df_kilo_capita_c


df_kilo_capita_c = df_kilo_capita_c.rename(columns={'NACE_R2 (Labels)':'country'})
# Convert column names to lowercase, replace spaces with underscores, and replace parentheses with underscores
df_kilo_capita_c.columns = [col.lower().replace(' ', '_').replace('(', '@').replace(')', '@') for col in df_kilo_capita_c.columns]


df_kilo_capita_c['id'] = df_kilo_capita_c.apply(lambda row: f'waste_cat_{row["country"].lower()}{row["year"]}', axis=1)
# Assuming 'df_kilo_capita_c' is your DataFrame and 'column_to_move' is the name of the column you want to move to the first position
column_to_move = 'id'   # Adjust this to the actual name of the column you want to move

# Extract the column you want to move
moved_column = df_kilo_capita_c[column_to_move]

# Drop the column from its original position
df_kilo_capita_c = df_kilo_capita_c.drop(columns=[column_to_move])

# Insert the column at the first position
df_kilo_capita_c.insert(0, column_to_move, moved_column)
# Insert the column at the first position



#cleaned
# df_kilo_capita_c.to_csv('../source/check.csv',index=False)

In [80]:
# Slice rows from index 10 to 38 of df3_food_waste and reset the index, dropping the old index
df_tonne = df3_food_waste[12:39].reset_index(drop=True)

# Rename the columns of df_tonne using the values from the 9th row of df3_food_waste
df_tonne = df_tonne.rename(columns=df3_food_waste.iloc[9])

# Add a new column named 'year' to df_tonne and set all values in that column to 2020
df_tonne['year'] = 2020

# Slice rows from index 10 to 38 of df4_food_waste and reset the index, dropping the old index
df_tonne2 = df4_food_waste[12:39].reset_index(drop=True)

# Rename the columns of df_tonne2 using the values from the 9th row of df4_food_waste
df_tonne2 = df_tonne2.rename(columns=df4_food_waste.iloc[9])

# Add a new column named 'year' to df_tonne2 and set all values in that column to 2021
df_tonne2['year'] = 2021

# Concatenate df_tonne and df_tonne2 vertically, stack them on top of each other, and reset the index, dropping the old index
df_tonne = pd.concat([df_tonne, df_tonne2]).reset_index(drop=True)


# First, find the column names that are NaN
nan_columns = df_tonne.columns[df_tonne.columns.isna()]

# Then drop these columns
df_tonne_c = df_tonne.drop(columns=nan_columns)






df_tonne_c = df_tonne_c.rename(columns={'NACE_R2 (Labels)':'country'})
# Convert column names to lowercase, replace spaces with underscores, and replace parentheses with underscores
df_tonne_c.columns = [col.lower().replace(' ', '_').replace('(', '@').replace(')', '@') for col in df_tonne_c.columns]


df_tonne_c['id'] = df_tonne_c.apply(lambda row: f'waste_cat_{row["country"].lower()}{row["year"]}', axis=1)
# Assuming 'df_tonne_c' is your DataFrame and 'column_to_move' is the name of the column you want to move to the first position
column_to_move = 'id'   # Adjust this to the actual name of the column you want to move

# Extract the column you want to move
moved_column = df_tonne_c[column_to_move]

# Drop the column from its original position
df_tonne_c = df_tonne_c.drop(columns=[column_to_move])

# Insert the column at the first position
df_tonne_c.insert(0, column_to_move, moved_column)

#claned
# df_tonne_c.to_csv('../source/check.csv',index=False)




In [85]:
df_composition
# Convert column names to lowercase, replace spaces with underscores, and replace parentheses with underscores
df_composition.columns = [col.lower().replace(' ', '_').replace('(', '@').replace(')', '@') for col in df_composition.columns]
df_composition

#claned ??
# df_composition.to_csv('../source/check.csv',index=False)


In [95]:
df_bq_keywords_eu = df_bq_keywords[df_bq_keywords['geoareaname'].isin(eu_country)].reset_index(drop=True)

#DROP EMPTY COLUMN
df_bq_keywords_eu = df_bq_keywords_eu.drop(columns=['cities','education_level','freq','hazard_type','ihr_capacity','level_status','location','migratory_status','mode_of_transportation',
'name_of_international_institution','name_of_non_communicable_disease','sex','tariff_regime_status','type_of_mobile_technology','type_of_occupation','type_of_skill','type_of_speed'])


#clean ??
# df_bq_keywords_eu.to_csv('../source/check.csv',index=False)


In [108]:

df_api_12['goal'] = df_api_12['goal'].apply(lambda x: x.replace('[','').replace(']','').replace("'", ''))
df_api_12['target'] = df_api_12['target'].apply(lambda x: x.replace('[','').replace(']','').replace("'", ''))
df_api_12['indicator'] = df_api_12['indicator'].apply(lambda x: x.replace('[','').replace(']','').replace("'", ''))
df_api_12['attributes'] = df_api_12['attributes'].apply(lambda x: x.replace('{','').replace('}','').replace("'", ''))
df_api_12['dimensions'] = df_api_12['dimensions'].apply(lambda x: x.replace('{','').replace('}','').replace("'", ''))
df_api_12['footnotes'] = df_api_12['footnotes'].apply(lambda x: x.replace('[','').replace(']','').replace("'", ''))

# df_api_12['value'] = df_api_12['value'].apply(lambda x: float(x))
df_api_12['value'] = df_api_12['value'].apply(lambda x:int(x))

df_12_eu = df_api_12[df_api_12['geoAreaName'].isin(eu_country)].reset_index(drop=True)

df_12_eu.columns = [col.lower().replace(' ', '_').replace('(', '@').replace(')', '@') for col in df_12_eu.columns]

#DROP EMPTY COLUMN
df_12_eu = df_12_eu.drop(columns=['time_detail','timecoverage','upperbound','lowerbound','geoinfourl'])

#clean ??
df_12_eu.to_csv('../source/check.csv',index=False)






In [113]:

df_api_12_3['goal'] = df_api_12_3['goal'].apply(lambda x: x.replace('[','').replace(']','').replace("'", ''))
df_api_12_3['target'] = df_api_12_3['target'].apply(lambda x: x.replace('[','').replace(']','').replace("'", ''))
df_api_12_3['indicator'] = df_api_12_3['indicator'].apply(lambda x: x.replace('[','').replace(']','').replace("'", ''))
df_api_12_3['attributes'] = df_api_12_3['attributes'].apply(lambda x: x.replace('{','').replace('}','').replace("'", ''))
df_api_12_3['dimensions'] = df_api_12_3['dimensions'].apply(lambda x: x.replace('{','').replace('}','').replace("'", ''))
df_api_12_3['footnotes'] = df_api_12_3['footnotes'].apply(lambda x: x.replace('[','').replace(']','').replace("'", ''))
df_12_3_eu = df_api_12_3[df_api_12_3['geoAreaName'].isin(eu_country)].reset_index(drop=True)
df_12_3_eu.columns = [col.lower().replace(' ', '_').replace('(', '@').replace(')', '@') for col in df_12_3_eu.columns]

#DROP EMPTY COLUMN
df_12_3_eu = df_12_3_eu.drop(columns=['time_detail','timecoverage','upperbound','lowerbound','baseperiod','geoinfourl'])

#clean ??
df_12_3_eu.to_csv('../source/check.csv',index=False)



In [142]:
#clean
df_top_fruit_veg_wasted

,top_fruit_veg_wasted
0,Salad leaves (bagged)
1,Bananas
2,Lettuce (whole)
3,Cucumber
4,Tomatoes
5,Carrots
6,Mushrooms
7,Potato
8,Grapes
9,Strawberries
